In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
def print_result(result: int, solver: pywraplp.Solver, 
                 delta: dict, I: set, J: set) -> None:
    
    if result == solver.OPTIMAL:
        print("Optimal solution found.")
    
    obj = solver.Objective().Value()
    print(f"obj = {round(obj, 2)}")
    
    I = list(I)
    I.sort()
    
    for i in list(I):
        for j in J:
            delta = delta_ij[i][j]
            
            if delta_ij[i][j].SolutionValue() == 1:
                print(f"Department {i} is located in {j}")
            else:
                pass
    
    return None

## Problems

These problems are designed to show some typical MILP problems. The problem focuses on a typical business problem called decentralisation, and is taken from the textbook [Model Building in Mathematical Programming by H.P. Williams](https://www.wiley.com/en-gb/Model+Building+in+Mathematical+Programming%2C+5th+Edition-p-9781118443330). Chapter 12 of that textbook has many more interesing MILP problems if people want to do any further reading. 

The problem has been broken into two parts. The first is a relatively simple MILP about the location of various facilities. The second adds an additional layer of complexity to the problem, and highlights one of the key transforms we can do when introducing integers into the problem.

When completing these exercises, remember some of the key steps to solving a mathematical program:

1. Describe the **constraints** and **objectives** verbally.
2. Describe the **entities** as sets.
3. Describe any **parameters** as sets.
4. Define the **decision variables** that seem sufficient to solve the problem.
5. Write the **objective function** using the parameters and decision variables.
6. Write the **constraints** using sets and decision variables.
7. Iterate & repeat until happy.

### Decentralisation 1

A large company wishes to move some of its departments out of London. There are benefits to be derived from doing this (cheaper housing, government incentives, easier recruitment, etc.), which have been costed.

The company comprises five departments (A, B, C, D and E). The possible cities for relocation are Bristol and Brighton, or a department may be kept in London. None of these cities (including London) may be the location for more than three of the departments.

Benefits to be derived from each relocation are given (in thousands of pounds per year) as follows:

| City     | A  | B  | C  | D  | E  |
|----------|----|----|----|----|----|
| Bristol  | 10 | 15 | 10 | 20 | 5  |
| Brighton | 10 | 20 | 15 | 15 | 15 |

In [ ]:
solver = pywraplp.Solver("decentralisation", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

### 1. Describe the rules and constraints verbally

### 2. Describe the entities as sets

### 3. Describe related parameters as sets

These are the associated benefits. They can be easily described as a matrix of $i$ and $j$ pairs. This time, we also have two additional matrices to describe the between-department volumes of communication and the communication costs between cities. 

In [ ]:
# benefits matrix
V_ij = {"A": {"bristol": 10, "brighton": 10, "london": 0},
        "B": {"bristol": 15, "brighton": 20, "london": 0}, 
        "C": {"bristol": 10, "brighton": 15, "london": 0}, 
        "D": {"bristol": 20, "brighton": 15, "london": 0}, 
        "E": {"bristol": 5, "brighton": 15, "london": 0}}

### 4. Decision variables

### 5. Objective function

### 6. Constraints

In [ ]:
# run solver
result = solver.Solve()
print_result(result=result, solver=solver, delta=delta_ij, I=I, J=J)

### Decentralisation 2

It has been suggested that there will be greater costs of communication between departments cause by relocation across multiple sites. These have also been costed for all possible locations of each department.

Communication costs are of the form $C_{ik} \cdot D_{jl}$, where $C_{ik}$ is the quantity of communication between departments $i$ and $k$ per year and $D_{jl}$ is the cost per unit of communication between cities $j$ and $l$. $C_{ik}$ and $D_{jl}$ are given by the following tables:

|   | A | B   | C   | D   | E   |
|---|---|-----|-----|-----|-----|
| A |   | 0.0 | 1.0 | 1.5 | 0.0 |
| B |   |     | 1.4 | 1.2 | 0.0 |
| C |   |     |     | 0.0 | 2.0 |
| D |   |     |     |     | 0.7 |

|          | Bristol | Brighton | London |
|----------|---------|----------|--------|
| Bristol  |  5.0    | 14.0     | 13.0   |
| Brighton |         |   5.0    | 9.0    |
| London   |         |          |  10.0  |

Where should each department be located so as to minimise overall yearly cost?

#### Hint:

If we want to have a binary decision variable, $\gamma$ that is 1 iff $\delta_1 = 1$ and $\delta_2 = 1$, we can introduce the following constraints:

$\gamma - \delta_1 \leq 0$

$\gamma - \delta_2 \leq 0$

$\delta_1 + \delta_2 - \gamma \leq 1$

In [ ]:
solver = pywraplp.Solver("decentralisation-2", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

### 1. Describe the rules and constraints verbally

### 2. Describe the entities as sets

### 3. Describe related parameters as sets

These are the associated benefits. They can be easily described as a matrix of $i$ and $j$ pairs. This time, we also have two additional matrices to describe the between-department volumes of communication and the communication costs between cities. 

In [ ]:
# benefits matrix
V_ij = {"A": {"bristol": 10, "brighton": 10, "london": 0},
        "B": {"bristol": 15, "brighton": 20, "london": 0}, 
        "C": {"bristol": 10, "brighton": 15, "london": 0}, 
        "D": {"bristol": 20, "brighton": 15, "london": 0}, 
        "E": {"bristol": 5, "brighton": 15, "london": 0}}

# communication volume matrix
C_ik = {"A": {"A": 0.0, "B": 0.0, "C": 1.0, "D": 1.5, "E": 0.0},
        "B": {"A": 0.0, "B": 0.0, "C": 1.4, "D": 1.2, "E": 0.0},
        "C": {"A": 1.0, "B": 1.4, "C": 0.0, "D": 0.0, "E": 2.0},
        "D": {"A": 1.5, "B": 1.2, "C": 0.0, "D": 0.0, "E": 0.7},
        "E": {"A": 0.0, "B": 0.0, "C": 2.0, "D": 0.7, "E": 0.0}}

# cost matrix
D_jl = {"bristol": {"bristol": 5.0, "brighton": 14.0, "london": 13.0},
        "brighton": {"bristol": 14.0, "brighton": 5.0, "london": 9.0},
        "london": {"bristol": 13.0, "brighton": 9.0, "london": 10.0}}

### 4. Decision variables

### 5. Objective function

### 6. Constraints

In [ ]:
# run solver
result = solver.Solve()
print_result(result=result, solver=solver, delta=delta_ij, I=I, J=J)